In [2]:
import os
import sys
from IPython.display import clear_output
import logging

class MemoryOptimizedLogger:
    def __init__(self, max_lines=100):
        self.max_lines = max_lines
        self.line_count = 0
        
    def log(self, message):
        print(message, flush=True)
        self.line_count += 1
        if self.line_count >= self.max_lines:
            clear_output(wait=True)
            self.line_count = 0

logger = MemoryOptimizedLogger()

# Configuración
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(project_root)

import torch
from qgis.core import QgsApplication, QgsRasterLayer
from qgis.analysis import QgsNativeAlgorithms
import processing

input_file = r"E:\work\ProyectoCatasPro\06 ORTOFOTOS - copia\Ortofoto_Itagui_2021.ecw"
output_dir = r"E:\entrenamientomaquina\tiles generados\2021"

def process_tiles():
    try:
        logger.log("Iniciando procesamiento...")
        
        # CUDA check
        logger.log(f"CUDA disponible: {torch.cuda.is_available()}")
        if torch.cuda.is_available():
            logger.log(f"GPU: {torch.cuda.get_device_name(0)}")
            torch.cuda.empty_cache()

        # QGIS init
        qgis_prefix = "C:/OSGeo4W/apps/qgis"
        QgsApplication.setPrefixPath(qgis_prefix, True)
        qgs = QgsApplication([], False)
        qgs.initQgis()
        QgsApplication.processingRegistry().addProvider(QgsNativeAlgorithms())

        logger.log("Cargando raster...")
        raster_layer = QgsRasterLayer(input_file, "ortofoto")
        if not raster_layer.isValid():
            raise ValueError("Capa raster inválida")

        params = {
            'INPUT': raster_layer,
            'ZOOM_MIN': 0,
            'ZOOM_MAX': 18,
            'DPI': 96,
            'TILE_FORMAT': 0,
            'QUALITY': 75,
            'METATILESIZE': 4,
            'TILE_WIDTH': 256,
            'TILE_HEIGHT': 256,
            'TMS_CONVENTION': False,
            'OUTPUT_DIRECTORY': output_dir
        }

        logger.log("Generando tiles...")
        processing.run("gdal:tilesxyz", params)
        logger.log(f"Tiles generados en: {output_dir}")

    except Exception as e:
        logger.log(f"Error: {str(e)}")
    finally:
        qgs.exitQgis()
        torch.cuda.empty_cache()
        logger.log("Proceso finalizado")

process_tiles()

ModuleNotFoundError: No module named 'torch'